In [6]:
library(dplyr)
library(data.table)
library(tidyr)
library(dtplyr)
library(stringr)
library(ggplot2)

In [7]:
n = str_split("N	Nall	p	mean	median	min	max	sd",pattern = "\t")[[1]]
nv = str_split("N	Nall	mean	median	min	max	sd",pattern = "\t")[[1]]
df = fread(snakemake@input[[1]], col.names = n)

In [8]:
dfl <- pivot_longer(data = df,cols = unlist(nv), names_to = "measure", values_to = "values" ) %>%
    as.data.frame()

In [9]:
ggplot(data=dfl, aes(x=p, y=values)) + geom_line() + facet_grid(rows = vars(measure), scales = "free_y")

In [10]:
q95 = quantile(df$N, 0.95)[[1]]
df2 <- df %>%
    filter(N>=q95) %>%
    as.data.frame()
dfl2 <- pivot_longer(data = df2,cols = unlist(nv), names_to = "measure", values_to = "values" ) %>%
    as.data.frame()
ggplot(data=dfl2, aes(x=p, y=values)) + geom_line() + facet_grid(rows = vars(measure), scales = "free_y")

In [11]:
df95 <- df %>%
    filter(N >= q95) %>%
    arrange(sd) %>%
    as.data.frame()

In [12]:
p = df95$p[[1]]
f_cluster = paste(snakemake@params$clusters,p,"_ini.tsv",sep="")
df_cluster = fread(f_cluster)
dfo <- df_cluster %>%
    group_by(ClusterNumber) %>%
    summarise(C=list(SequenceName)) %>%
    as.data.frame()

In [13]:
cldf <- fread(snakemake@input$merged)

df <- fread(f_cluster)

data <- fread(snakemake@input$data, select = c("taxon","month"))

cldf2 <- cldf %>%
    left_join(data, by = c("Member"="taxon")) %>%
    mutate(Cluster = str_replace_all(Cluster,pattern = "'",replacement = "")) %>%
    left_join(df, by=c("Cluster"="SequenceName")) %>%
    as.data.frame()

mnths = unique(sort(data$month))
m1 = mnths[[1]]
m2 = mnths[[length(mnths)]]

df2 <- cldf2 %>%
    filter(month == m1 | month ==m2)
df3 <- df2  %>%
    group_by(ClusterNumber, month) %>%
    summarise(N=n()) %>%
    tidyr::pivot_wider(names_from = month, values_from = N) %>%
    na.omit() %>%
    rowwise() %>%
    mutate(total_nmb = sum(c_across(starts_with("20")))) %>%
    as.data.frame()
df3$ratio <- df3[,2] / df3[,3]
good <- df3 %>%
    filter(total_nmb >= 300) %>%
    filter(ratio < 1, ratio > 0.1, ClusterNumber != -1)
good

In [14]:
fwrite(snakemake@output$best, x = good)

In [17]:
#dfo$ClusterNumber
dir.create(snakemake@output$clusters)
good$ClusterNumber
dfoo <- dfo %>%
    filter(ClusterNumber %in% good$ClusterNumber)
ct = 0
for (l in dfoo$C[1:10]) {
    ct = ct + 1
    outf = paste(snakemake@output$clusters,"/","cl",ct,sep="")
    dft <- cldf %>%
        filter(Cluster %in% l) %>%
        select(Member) %>%
        as.data.frame()
    print(outf)
    fwrite(x = dft,file = outf, col.names = F, row.names = F)
}